In this notebook, we will be preparing anything that might be needed for future steps, such as co-expression networks and integrating different data sources (both from the challenge and from elsewhere).

### Co-Expression Network

--- WRITE SOME INTRO! --- # TODO

In [1]:
import dask.dataframe as dd
import pandas as pd

target_cell = "B cells"

# multiome_obs_meta = pd.read_csv("../data/multiome_obs_meta.csv")
# obs_ids = multiome_obs_meta[multiome_obs_meta["cell_type"] == target_cell]["obs_id"].to_list()

# multiome_var_meta = pd.read_csv("../data/multiome_var_meta.csv")
# gene_names = multiome_var_meta[multiome_var_meta["feature_type"] == "Gene Expression"]["location"].to_list()

adata_obs_meta = pd.read_csv("../data/adata_obs_meta.csv")
obs_ids = adata_obs_meta[adata_obs_meta["col"] == 3]["obs_id"].to_list()

In [7]:
adata_obs_meta[adata_obs_meta["col"] == 3]

,obs_id,library_id,plate_name,well,row,col,cell_id,donor_id,cell_type,sm_lincs_id,sm_name,SMILES,dose_uM,timepoint_hr,control
19,000683e13aa73f7f,library_44,plate_1,A3,A,3,PBMC,donor_2,T cells CD8+,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
23,0008106bb52a931d,library_19,plate_2,H3,H,3,PBMC,donor_0,B cells,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
37,000a41a4580023f5,library_43,plate_2,G3,G,3,PBMC,donor_0,NK cells,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
38,000a59643eff306a,library_47,plate_2,F3,F,3,PBMC,donor_0,T cells CD4+,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
39,000a7dc7f1cba897,library_7,plate_4,H3,H,3,PBMC,donor_2,Myeloid cells,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240045,fff6f64b0cd2160b,library_24,plate_5,G3,G,3,PBMC,donor_1,T cells CD8+,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
240055,fff97cc3be63ebc0,library_31,plate_4,E3,E,3,PBMC,donor_2,NK cells,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
240067,fffb8bd514c27106,library_45,plate_5,A3,A,3,PBMC,donor_1,B cells,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
240073,fffc885a62e5dab7,library_27,plate_0,G3,G,3,PBMC,donor_0,T cells CD4+,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True


In [8]:
from pyarrow.parquet import ParquetFile

target_adata_df = pd.DataFrame()
batch_num = 10

adata_file = ParquetFile("../data/adata_train.parquet")
batch_group_size = adata_file.metadata.num_rows // batch_num
for batch in adata_file.iter_batches(batch_size=batch_group_size):
    adata_batch = batch.to_pandas()
    adata_batch = adata_batch[adata_batch["obs_id"].isin(obs_ids)]
    target_adata_df = pd.concat([target_adata_df, adata_batch], axis=0, ignore_index=True)

In [12]:
target_adata_df["obs_id"].unique().shape

(28462,)

In [13]:
target_data_df = target_adata_df.pivot_table("normalized_count", "obs_id", "gene")

In [21]:
target_data_df.describe()

gene,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
count,365.000000,950.000000,2734.000000,2360.000000,152.000000,71.000000,2132.000000,751.000000,3151.000000,14015.000000,...,1551.000000,1710.000000,570.000000,339.000000,572.000000,1900.000000,9.000000,4001.000000,7885.000000,4023.000000
mean,4.950278,4.738525,4.842949,5.398524,5.381525,4.194634,4.726670,4.679955,4.757981,5.345509,...,4.510811,4.491884,4.386094,4.944024,4.839680,4.672655,5.090018,4.721535,5.007226,4.791031
std,0.620414,0.668572,0.745197,0.627675,0.380030,0.459680,0.689722,0.700162,0.683998,0.700825,...,0.688021,0.664819,0.644947,0.610912,0.664701,0.692819,0.363988,0.658892,0.584179,0.697022
min,3.547446,3.536149,3.537801,3.536926,3.592938,3.545124,3.536149,3.539976,3.536117,3.536246,...,3.536117,3.536246,3.541895,3.546170,3.544242,3.536926,4.378929,3.536699,3.537769,3.536699
25%,4.711703,4.064604,4.132125,5.148252,5.217587,3.919624,4.031493,3.977552,4.079128,4.979088,...,3.896715,3.926193,3.869470,4.642245,4.147417,4.003076,4.971235,4.104736,4.651582,4.118081
50%,5.166366,4.973517,4.915393,5.464024,5.429976,4.011186,4.921189,4.800004,4.924032,5.348083,...,4.295935,4.301630,4.115676,5.093564,5.056906,4.739515,5.037249,4.795533,5.081722,4.932883
75%,5.340409,5.281850,5.451523,5.726687,5.591561,4.421622,5.280194,5.264482,5.292027,5.820230,...,5.145873,5.070022,5.020258,5.356343,5.345952,5.269126,5.326663,5.243272,5.402066,5.341547
max,6.291455,6.135656,7.143509,7.279147,6.541090,5.670085,6.637514,6.334979,6.545592,7.563301,...,6.336010,6.428798,6.204578,6.086952,6.530436,6.600162,5.662766,6.799119,6.886690,6.920478


In [24]:
import PyWGCNA
exp = PyWGCNA.WGCNA(name=target_cell, species="Homo sapiens", geneExp=target_data_df.fillna(0))
exp.geneExpr.to_df().head(5)

gene,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
obs_id,,,,,,,,,,,,,,,,,,,,,
000683e13aa73f7f,0.0,0.0,0.0,5.73634,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
0008106bb52a931d,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
000a41a4580023f5,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,5.238042,0.0
000a59643eff306a,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,4.921865,...,0.000000,0.0,4.921865,0.0,0.0,0.000000,0.0,0.000000,5.611362,0.0
000a7dc7f1cba897,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,4.240418,...,4.240418,0.0,0.000000,0.0,0.0,4.240418,0.0,4.240418,4.240418,0.0


In [25]:
exp.preprocess()

Pre-processing...
	Detecting genes and samples with too many missing values...


In [ ]:
exp.geneExpr.to_df().head(5)

In [2]:
from pyarrow.parquet import ParquetFile

target_adata_df = pd.DataFrame()
batch_num = 10

adata_file = ParquetFile("../data/adata_train.parquet")
batch_group_size = adata_file.num_row_groups // batch_num
for batch in range(batch_num):
    if batch < batch_num - 1:
        s = range(batch*batch_group_size, (batch+1)*batch_group_size)
    else:
        s = range(batch*batch_group_size, adata_file.num_row_groups)
    adata_batch = adata_file.read_row_groups(s).to_pandas()
    adata_batch = adata_batch[adata_batch["obs_id"].isin(obs_ids)]
    target_adata_df = pd.concat([target_adata_df, adata_batch], axis=0, ignore_index=True)

In [23]:
target_data_df.notna().sum(axis=1)

obs_id
000683e13aa73f7f    1204
0008106bb52a931d    1052
000a41a4580023f5    1927
000a59643eff306a    1991
000a7dc7f1cba897    3580
                    ... 
fff6f64b0cd2160b    1609
fff97cc3be63ebc0    1355
fffb8bd514c27106    2382
fffc885a62e5dab7    2244
fffcb76d7d937647    4568
Length: 28462, dtype: int64

416442312

In [46]:
from pyarrow.parquet import ParquetFile

target_multiome_df = pd.DataFrame()
batch_num = 10

multiome_file = ParquetFile("../data/multiome_train.parquet")
batch_group_size = multiome_file.num_row_groups // batch_num
for batch in range(batch_num):
    if batch < batch_num - 1:
        s = range(batch*batch_group_size, (batch+1)*batch_group_size)
    else:
        s = range(batch*batch_group_size, multiome_file.num_row_groups)
    multiome_batch = multiome_file.read_row_groups(s).to_pandas()
    multiome_batch = multiome_batch[multiome_batch["obs_id"].isin(obs_ids)]
    # multiome_batch = multiome_batch[multiome_batch["location"].isin(gene_names)]
    target_multiome_df = pd.concat([target_multiome_df, multiome_batch], axis=0, ignore_index=True)

In [81]:
target_cell = "B cells"

adata_obs_meta = pd.read_csv("../data/adata_obs_meta.csv")
obs_ids = adata_obs_meta[adata_obs_meta["col"] == 3]["obs_id"].to_list()

In [88]:
x.fillna(0)

gene,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
obs_id,,,,,,,,,,,,,,,,,,,,,
000683e13aa73f7f,0.0,0.000000,0.000000,5.736340,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
0008106bb52a931d,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
000a41a4580023f5,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,5.238042,0.0
000a59643eff306a,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,4.921865,...,0.000000,0.000000,4.921865,0.0,0.0,0.000000,0.0,0.000000,5.611362,0.0
000a7dc7f1cba897,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,4.240418,...,4.240418,0.000000,0.000000,0.0,0.0,4.240418,0.0,4.240418,4.240418,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff6f64b0cd2160b,0.0,5.212115,0.000000,5.212115,0.0,0.0,0.000000,0.0,0.0,5.902534,...,0.000000,0.000000,0.000000,0.0,0.0,5.212115,0.0,0.000000,0.000000,0.0
fff97cc3be63ebc0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,6.239318,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
fffb8bd514c27106,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0


In [91]:
c = x[1:10].corr()
c

MemoryError: Unable to allocate 3.32 GiB for an array with shape (21122, 21122) and data type float64

In [53]:
# Transform dataframe --> columns as genes and rows as samples...
x = target_multiome_df[target_multiome_df["location"].isin(gene_names)].pivot_table("normalized_count", "obs_id", "location")

In [86]:
target_multiome_df

,obs_id,location,count,normalized_count
0,000225c1151ab841,AAMP,1,6.320659
1,000225c1151ab841,AASS,1,6.320659
2,000225c1151ab841,ABCC11,1,6.320659
3,000225c1151ab841,ABCC2,1,6.320659
4,000225c1151ab841,ABR,1,6.320659
...,...,...,...,...
34311166,ffe93b989e73959e,chrX:63754752-63755597,1,2.770587
34311167,ffe93b989e73959e,chrX:64205465-64206345,2,3.048007
34311168,ffe93b989e73959e,chrX:6738626-6739524,2,8.954666
34311169,ffe93b989e73959e,chrX:78754674-78755560,2,9.734200


In [70]:
(x["A1BG"].notna() & x["ZXDC"].notna()).sum()

51